In [2]:
import numpy as np
from datetime import datetime

from aiida import load_profile
from aiida.orm import *

from qe_tools import CONSTANTS as C

from ase.io import write
from ase import units
from ase.calculators.singlepoint import SinglePointCalculator

load_profile()

Profile<uuid='f1b179e2a9c84e01809b87556826d82b' name='quicksetup'>

In [28]:
# group_label = 'CsPbI3/SSCHA/250K'

qb = QueryBuilder()
# qb.append(Group, filters={'label': group_label}, tag='g')
# qb.append(WorkChainNode, filters={'attributes.exit_status': 0}, with_group='g', tag='wc')
qb.append(
    WorkChainNode,
    filters={
        'attributes.exit_status': 0,
        'attributes.process_label':'PwBaseWorkChain',
        'ctime': {'>=': datetime(2024, 4, 4)},  
    },
    tag='wc',
)
qb.append(
    TrajectoryData,
    # filters=(Node.fields.attributes.symbols == ['Si']),
    filters={
        'attributes.symbols': {'contains': ['Si'], 'shorter': 17, 'longer': 15},
    },
    with_incoming='wc',
)

qb.count()

91

In [29]:
results = qb.all(flat=True)

In [30]:
results[0].base.attributes.all

{'symbols': ['Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si',
  'Si'],
 'array|cells': [1, 3, 3],
 'array|steps': [1],
 'array|energy': [1],
 'array|forces': [1, 16, 3],
 'array|stress': [1, 3, 3],
 'array|energy_xc': [1],
 'array|positions': [1, 16, 3],
 'array|total_force': [1],
 'array|energy_ewald': [1],
 'array|fermi_energy': [1],
 'array|scf_accuracy': [9],
 'array|energy_hartree': [1],
 'array|scf_iterations': [1],
 'array|energy_accuracy': [1],
 'array|energy_smearing': [1],
 'array|energy_threshold': [1],
 'array|atomic_species_name': [16],
 'array|energy_one_electron': [1]}

In [36]:
C.ry_to_ev, units.Ry

(13.6056917253, 13.605693012183622)

In [31]:
stress_units = (-1 * units.Ry / units.Bohr**3)/(C.ry_si / C.bohr_si**3 / 10**9) # convention as in ASE (sign and eV/Ang^3)

filename = './dataset-sscha.xyz'

for res in results:
    index = 0
    atoms = res.get_step_structure(index).get_ase()
    energy = res.get_array('energy')[index]
    s = res.get_array('stress')[index]

    calc = SinglePointCalculator(atoms)

    calc.results = {
        'energy': energy,
        'free_energy': energy,
        'forces': res.get_array('forces')[index],
        'stress': stress_units*np.array([s[0,0],s[1,1],s[2,2],s[1,2],s[0,2],s[0,1]]),
    }

    atoms.calc = calc

    write(filename, atoms, format='extxyz', append=True)
    # break

In [22]:
s*stress_units

array([[ 0.03541242, -0.01678025, -0.01865329],
       [-0.01678025,  0.04417991, -0.01289093],
       [-0.01865329, -0.01289093,  0.04192492]])

In [9]:
from qe_tools import CONSTANTS as C

C.ry_to_ev / C.bohr_to_ang

25.711031209285363

In [10]:
-0.12190370*25.711031209285363

-3.1342698352273604

In [6]:
res.get_array('forces')[index]

array([[ 5.33819603e-01, -3.13426977e+00,  2.21529350e+00],
       [-3.61151040e-01,  1.14138113e-01,  3.71801765e-02],
       [-1.29917331e-01, -4.65262292e-02,  4.80323948e-04],
       [ 6.49981407e-03, -5.67746845e-02, -5.44976344e-01],
       [ 1.43491153e+00, -1.29718723e-02, -1.85130988e+00],
       [ 6.30538141e+00, -2.56791248e+00,  7.08259788e+00],
       [-5.78140040e-01, -2.29837349e-01,  4.43329029e-01],
       [ 9.42794295e-01, -7.23102748e-02,  6.58932930e-02],
       [ 1.03831906e-01,  5.36502991e-03,  1.89444206e-01],
       [-6.22099867e+00,  2.91243849e+00, -7.01624532e+00],
       [-1.35660162e+00,  2.46259856e+00, -9.54574037e-01],
       [-1.32330763e-01, -8.08972545e-01,  7.27709766e-01],
       [-6.03900509e-01,  3.05007627e+00, -2.38370475e+00],
       [-1.99602495e-02, -5.40431055e-01,  9.21258153e-01],
       [ 2.50064946e-01,  1.22705865e+00, -6.68718486e-01],
       [-1.21055158e-01,  1.98872765e-01, -2.05141292e-01],
       [-2.16071821e-02, -5.25956510e-04

In [8]:
res.creator

/home/bastonero/.conda/envs/aiida/lib/python3.9/site-packages/aiida/storage/psql_dos/backend.py:270: SAWarning: Object of type <DbNode> not in session, add operation along 'DbUser.dbnodes' will not proceed
  with session.begin_nested() as savepoint:


<CalcJobNode: uuid: 394bf45a-b992-495c-87d4-75fec8b60b81 (pk: 4167626) (aiida.calculations:quantumespresso.pw)>